In [ ]:




[수도권의 일자리 분산 - Why]





In [ ]:
Insight
-  상/하선 혼잡도 120% 넘는 구간이 40곳이 존재함을 확인함으로써, 수도권 지하철의 혼잡도가 높다고 판단된다.
-  출/퇴근시간에 혼잡도가 높은 사실과 업무 지구 관련 지역(업무지구, 환승)에서 혼잡도가 높아지는 현상을 고려하였을때,
혼잡도가 높은 주요 이유로 과다하게 일자리가 집중되어 있음을 꼽을 수 있다. 다시 말해, 일자리 분산을 통해 혼잡도를 낮출 수 있다고 판단된다.

Limitation
아래 한계를 비롯하여 지하철과 함께 대중교통의 주축인 버스 혼잡도 등 추가사항을 조사하지 않는 이유는,
이미 참조(외부) 자료에서 수도권 대중교통 혼잡도 문제를 밝히고 있다고 판단되며, 검증의 과정으로서 짧은 조사로 마친다.
- 각 호선은 노선 별로 정리되어 있는데, 상선의 8호선의 복정이나 하선의 8호선 송파와 같이 각 호선 끝에 위치한 역의 경우,
특이 사항(혼잡률이 100보다 낮은 보이지 않는 노선에서 탑승객 상승 역)이 존재할 수 있다. 
- 2호선은 제외되었다. (칼럼"상하구분"이 내/외선 구분된다.)

In [ ]:
지하철(1~8호선) 혼잡도

4Q 2025. "서울교통공사_지하철혼잡도정보". 공공데이터포털.

데이터 순서
평일, 토요일, 일요일

In [1]:
mykey = ''

In [2]:
import requests, json, pandas as pd

headers = {'Authorization': f'Infuser {mykey}'}
url = 'https://api.odcloud.kr/api/15071311/v1/uddi:9c79ebd1-20e1-4e97-8e6a-b65e890fbe85?page=1&perPage=2000'
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data['data'])
    print(df)             
    
    print("\n컬럼 목록:", df.columns.tolist())

else:
    print("API 호출 실패")

     00시00분 00시30분 10시00분 10시30분 11시00분 11시30분 12시00분 12시30분 13시00분 13시30분  \
0       5.6    1.4   41.4   46.8   36.5   34.3   32.8   46.4   38.3   31.9   
1       8.4    0.0   23.1   17.9   21.2   24.5   28.6   28.2   32.4   36.2   
2       6.8    1.5   35.4   42.3   34.5   31.2   30.4   42.6   35.9   30.0   
3       5.5    0.3   23.0   19.1   20.8   23.4   28.8   26.1   31.8   35.0   
4       7.7    1.4   31.9   37.5   30.7   27.9   29.2   36.5   34.0   30.2   
...     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
1666    0.4    0.0    8.3    8.1    8.6    8.3    9.8    9.7    9.1    9.5   
1667    0.0    0.0    4.5    4.8    4.7    4.9    5.9    6.9    7.0    8.1   
1668    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1669    0.0    0.0   30.2   27.6   26.6   26.3   28.4   28.6   27.2   25.6   
1670    0.0    0.0   11.5   10.7   11.4   11.5   11.0   13.6   14.1   14.8   

      ... 7시30분 8시00분 8시30분 9시00분 9시30분 상하구분   역번호 요일구분  출발역   

In [3]:
# 필요 칼럼만 추출 및 순서 조정
stringData = [43,42,39,41] # 역번호를 제외한 메타데이터: 호선, 출발역, 상하구분, 요일구분
timeData = [i for i in range(31,38,2)] + [i for i in range(2,29,2)]+ [0] # 정시 데이터만 확인 및 순서 조정
 
staging_df = df.iloc[:, stringData+timeData]

# 칼럼 시간 표현 변경(00시00분 -> 0, 2시00분 -> 2)
time_cols = [col for col in df.columns if '시00분' in col]

new_names = {}
for col in time_cols:
    hour_str = col.split('시')[0]          # "6", "7", ..., "00"
    hour = int(hour_str)                   # 문자열 → 정수
    new_names[col] = str(hour)             # "6", "0" 등 문자열로

staging_df = staging_df.rename(columns=new_names)
print(staging_df)

       호선  출발역 상하구분 요일구분     6     7     8     9    10    11  ...    15    16  \
0     1호선  서울역   상선   평일  20.8  37.4  72.3  62.5  41.4  36.5  ...  27.8  25.9   
1     1호선  서울역   하선   평일  11.2  22.9  41.4  29.6  23.1  21.2  ...  47.3  53.8   
2     1호선   시청   상선   평일  15.4  29.4  58.7  48.5  35.4  34.5  ...  29.5  28.5   
3     1호선   시청   하선   평일  10.4  21.7  35.3  29.8  23.0  20.8  ...  44.5  48.9   
4     1호선   종각   상선   평일  13.5  19.3  35.4  31.5  31.9  30.7  ...  32.1  32.7   
...   ...  ...  ...  ...   ...   ...   ...   ...   ...   ...  ...   ...   ...   
1666  8호선   수진   하선  일요일   3.8   3.8   6.2   9.6   8.3   8.6  ...   7.9   7.6   
1667  8호선   모란   상선  일요일   1.1   1.4   2.5   4.8   4.5   4.7  ...   8.6   9.4   
1668  8호선   모란   하선  일요일   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   
1669  8호선  남위례   상선  일요일   9.0  11.1  18.7  26.6  30.2  26.6  ...  23.9  21.8   
1670  8호선  남위례   하선  일요일   4.0   4.2   5.1   8.8  11.5  11.4  ...  17.8  23.1   

        17     18    19    

In [4]:
# 데이터들을 숫자(float) 타입으로 변환 - 분석하기 위함
# errors='coerce'를 쓰면 숫자가 아닌 이상한 문자열이 있어도 에러 대신 NaN으로 처리해 안전합니다.

time_cols = [str(i) for i in range(6, 24)] + ['0']

staging_df[time_cols] = staging_df[time_cols].apply(pd.to_numeric, errors='coerce')
print(staging_df)

       호선  출발역 상하구분 요일구분     6     7     8     9    10    11  ...    15    16  \
0     1호선  서울역   상선   평일  20.8  37.4  72.3  62.5  41.4  36.5  ...  27.8  25.9   
1     1호선  서울역   하선   평일  11.2  22.9  41.4  29.6  23.1  21.2  ...  47.3  53.8   
2     1호선   시청   상선   평일  15.4  29.4  58.7  48.5  35.4  34.5  ...  29.5  28.5   
3     1호선   시청   하선   평일  10.4  21.7  35.3  29.8  23.0  20.8  ...  44.5  48.9   
4     1호선   종각   상선   평일  13.5  19.3  35.4  31.5  31.9  30.7  ...  32.1  32.7   
...   ...  ...  ...  ...   ...   ...   ...   ...   ...   ...  ...   ...   ...   
1666  8호선   수진   하선  일요일   3.8   3.8   6.2   9.6   8.3   8.6  ...   7.9   7.6   
1667  8호선   모란   상선  일요일   1.1   1.4   2.5   4.8   4.5   4.7  ...   8.6   9.4   
1668  8호선   모란   하선  일요일   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   0.0   
1669  8호선  남위례   상선  일요일   9.0  11.1  18.7  26.6  30.2  26.6  ...  23.9  21.8   
1670  8호선  남위례   하선  일요일   4.0   4.2   5.1   8.8  11.5  11.4  ...  17.8  23.1   

        17     18    19    

In [ ]:
상선과 하선을 나눈 이유는, 데이터 가시성이다.

[상/하선 혼잡도 N별 조사]에서 확인할 수 있는 내용이다.
[하선 혼잡도 100 별 조사]의 1호선(상선: 서울역 ~ 시청, 이 호선만 방향이 반대이다.)을 제외하고는 아래에서 위로 보는 것이 상선 방향이다. (하선은 위 -> 아래) 
- 상선 방향
ex) 3호선 (교대 ~ 지축), 4호선 (남태령 ~ 성신여대)
- 하선 방향
ex) 3호선 (지축 ~ 안국), 4호선 (미아사거리 ~ 남태령)

In [5]:
# 1. 상선
df_upper = staging_df[staging_df['상하구분'] == '상선'].copy()   
# 2. 하선
df_lower = staging_df[staging_df['상하구분'] == '하선'].copy()

print(df_upper)
print(df_lower)

       호선    출발역 상하구분 요일구분     6     7     8     9    10    11  ...    15  \
0     1호선    서울역   상선   평일  20.8  37.4  72.3  62.5  41.4  36.5  ...  27.8   
2     1호선     시청   상선   평일  15.4  29.4  58.7  48.5  35.4  34.5  ...  29.5   
4     1호선     종각   상선   평일  13.5  19.3  35.4  31.5  31.9  30.7  ...  32.1   
6     1호선   종로3가   상선   평일  14.5  19.3  38.8  30.8  32.5  30.6  ...  35.8   
8     1호선   종로5가   상선   평일  10.9  14.5  29.6  26.2  26.8  28.3  ...  37.3   
...   ...    ...  ...  ...   ...   ...   ...   ...   ...   ...  ...   ...   
1661  8호선  단대오거리   상선  일요일   4.6   6.2  10.3  14.7  15.7  14.2  ...  15.0   
1663  8호선     신흥   상선  일요일   2.9   3.4   5.6   8.5   9.2   8.2  ...  12.4   
1665  8호선     수진   상선  일요일   1.7   2.3   3.5   6.5   6.2   6.1  ...  10.8   
1667  8호선     모란   상선  일요일   1.1   1.4   2.5   4.8   4.5   4.7  ...   8.6   
1669  8호선    남위례   상선  일요일   9.0  11.1  18.7  26.6  30.2  26.6  ...  23.9   

        16    17    18    19    20    21    22    23     0  
0     25.9  28

In [14]:
# [상/하선 혼잡도 N별 조사]에 공통으로 쓰이는 변수들을 따로 기록

# 대상 시간대 컬럼 리스트
time_cols = [str(i) for i in range(6, 24)] + ['0']
# 기본 정보 컬럼(호선, 출발역, 상하구분, 요일구분)은 유지해야 하므로 따로 지정합니다.
info_cols = ['호선', '출발역', '상하구분', '요일구분']

In [15]:
# [상선 혼잡도 N 별 조사]

# 설정값 N
N = 120

# 1. 행 필터링: 어느 한 시간대라도 N 이상인 '행'만 추출
df_upper_rows_filtered = df_upper[(df_upper[time_cols] >= N).any(axis=1)]

# 2. 열 필터링: 추출된 행들 중에서, 값이 모두 N 미만인 '시간대 컬럼'은 제외
# 각 시간대 컬럼별로 최대값이 N 이상인 컬럼만 선택
df_upper_active_time_cols = [col for col in time_cols if df_upper_rows_filtered[col].max() >= N]

# 3. 최종 결과: 필터링된 행 + (기본정보 컬럼 + 데이터가 있는 시간대 컬럼)
df_upper_final = df_upper_rows_filtered[info_cols + df_upper_active_time_cols]

# 결과 확인 
print(df_upper_final)

      호선     출발역 상하구분 요일구분      8     18
246  4호선     남태령   상선   평일  125.2   74.7
463  7호선     용마산   상선   평일   38.9  132.2
465  7호선      중곡   상선   평일   38.7  135.3
467  7호선      군자   상선   평일   41.9  137.1
469  7호선  어린이대공원   상선   평일   36.2  137.7
471  7호선    건대입구   상선   평일   37.8  137.6
475  7호선      청담   상선   평일   41.6  121.9
487  7호선      내방   상선   평일  130.0   50.0
489  7호선      이수   상선   평일  134.1   42.7
491  7호선      남성   상선   평일  126.8   36.6
511  7호선      철산   상선   평일  141.0   21.5
513  7호선   광명사거리   상선   평일  124.0   19.5
527  8호선    몽촌토성   상선   평일   41.9  128.7
529  8호선      잠실   상선   평일   45.0  127.7
531  8호선      석촌   상선   평일  110.5  120.1
533  8호선      송파   상선   평일  127.1  122.7
535  8호선    가락시장   상선   평일  116.8  120.6
537  8호선      문정   상선   평일  124.5  111.0
539  8호선      장지   상선   평일  125.8   60.9


In [ ]:
출/퇴근시간의 약 20개의 높은 혼잡도 역 존재

In [16]:
# 상선 혼잡도 100 별 조사

# 설정값 N
N = 100

# 1. 행 필터링: 어느 한 시간대라도 N 이상인 '행'만 추출
df_upper_rows_filtered = df_upper[(df_upper[time_cols] >= N).any(axis=1)]

# 2. 열 필터링: 추출된 행들 중에서, 값이 모두 N 미만인 '시간대 컬럼'은 제외
# 각 시간대 컬럼별로 최대값이 N 이상인 컬럼만 선택
df_upper_active_time_cols = [col for col in time_cols if df_upper_rows_filtered[col].max() >= N]

# 3. 최종 결과: 필터링된 행 + (기본정보 컬럼 + 데이터가 있는 시간대 컬럼)
df_upper_final = df_upper_rows_filtered[info_cols + df_upper_active_time_cols]

# 결과 확인 (퇴근 시간만 확인)
print(df_upper_final.iloc[:,[0,1,2,3,7,8,9]])

      호선         출발역 상하구분 요일구분     17     18     19
128  3호선          지축   상선   평일   46.3  106.7   77.7
138  3호선          홍제   상선   평일   88.8  104.7   73.6
140  3호선         무악재   상선   평일   95.6  113.1   76.9
142  3호선         독립문   상선   평일   97.4  115.4   79.2
144  3호선         경복궁   상선   평일  106.9  118.1   80.0
146  3호선          안국   상선   평일  103.3  112.5   79.7
148  3호선        종로3가   상선   평일   98.7  107.4   74.2
162  3호선         압구정   상선   평일   90.7  110.6   84.9
164  3호선          신사   상선   평일   86.2  103.7   74.1
170  3호선          교대   상선   평일   89.2  111.7   65.7
214  4호선      성신여대입구   상선   평일   91.2  107.2   86.7
216  4호선       한성대입구   상선   평일   97.7  114.1   95.1
218  4호선          혜화   상선   평일  101.8  114.4   94.8
220  4호선         동대문   상선   평일  105.7  118.6   97.9
222  4호선   동대문역사문화공원   상선   평일   99.8  115.5   96.1
224  4호선         충무로   상선   평일   85.3  100.7   65.7
246  4호선         남태령   상선   평일   62.6   74.7   40.2
270  5호선         오목교   상선   평일   75.9  103.3   63.7
272  5호선    

In [ ]:
[하선 혼잡도 100 별 조사]의 1호선(상선: 서울역 ~ 시청, 이 호선만 방향이 반대이다.)을 제외하고는 아래에서 위로 보는 것이 상선 방향이다. (하선은 위 -> 아래) 
- 상선 방향
ex) 3호선 (교대 ~ 지축), 4호선 (남태령 ~ 성신여대)
- 하선 방향
ex) 3호선 (지축 ~ 안국), 4호선 (미아사거리 ~ 남태령)

# 혼잡의 이유를 찾기 위해서는 퇴근 시간의 탑승객이 늘어나는 역을 주시해야 한다. (같은 맥락으로 하선에도 적용한다.)
- 주요 탑승객 상승 역(상승 폭 30)
4호선: 동대문역사공원(19) - 환승/업무지구
5호선: 여의도(17,18) - 업무지구
7호선: 강남 구청(17,18,19) - 업무지구
8호선: 문정(18) - 업무지구

In [17]:
# 하선 혼잡도 N 별 조사

# 설정값 N
N = 120

# 1. 행 필터링: 어느 한 시간대라도 N 이상인 '행'만 추출
df_lower_rows_filtered = df_lower[(df_lower[time_cols] >= N).any(axis=1)]

# 2. 열 필터링: 추출된 행들 중에서, 값이 모두 N 미만인 '시간대 컬럼'은 제외
# 각 시간대 컬럼별로 최대값이 N 이상인 컬럼만 선택
df_lower_active_time_cols = [col for col in time_cols if df_lower_rows_filtered[col].max() >= N]

# 3. 최종 결과: 필터링된 행 + (기본정보 컬럼 + 데이터가 있는 시간대 컬럼)
df_lower_final = df_lower_rows_filtered[info_cols + df_lower_active_time_cols]

# 결과 확인
print(df_lower_final)

      호선      출발역 상하구분 요일구분      8      9     18
1    1호선      서울역   하선   평일   41.4   29.6  126.3
139  3호선       홍제   하선   평일  130.8   78.9   22.3
141  3호선      무악재   하선   평일  129.2   82.8   22.7
143  3호선      독립문   하선   평일  129.4   84.5   23.9
145  3호선      경복궁   하선   평일  122.8   81.5   44.3
271  5호선      오목교   하선   평일  120.0   69.4   34.6
273  5호선       양평   하선   평일  123.2   71.8   38.2
279  5호선       신길   하선   평일  123.2   64.1   30.6
327  5호선       길동   하선   평일   58.9   20.5  121.2
462  7호선      사가정   하선   평일  124.4  121.4   30.8
464  7호선      용마산   하선   평일  128.1  123.4   29.5
466  7호선       중곡   하선   평일  138.5  128.7   33.4
468  7호선       군자   하선   평일  138.8  135.6   33.9
470  7호선   어린이대공원   하선   평일  139.8  135.2   38.6
510  7호선  가산디지털단지   하선   평일   32.8   11.5  141.3
512  7호선       철산   하선   평일   34.8   11.1  124.9
524  8호선       천호   하선   평일  152.5  120.1   49.1
526  8호선     강동구청   하선   평일  159.4  126.6   54.7
528  8호선     몽촌토성   하선   평일  150.8  128.0   60.8


In [ ]:
출/퇴근시간의 약 20개의 높은 혼잡도 역 존재

In [18]:
# 하선 혼잡도 100 별 조사

# 설정값 N
N = 100

# 1. 행 필터링: 어느 한 시간대라도 N 이상인 '행'만 추출
df_lower_rows_filtered = df_lower[(df_lower[time_cols] >= N).any(axis=1)]

# 2. 열 필터링: 추출된 행들 중에서, 값이 모두 N 미만인 '시간대 컬럼'은 제외
# 각 시간대 컬럼별로 최대값이 N 이상인 컬럼만 선택
df_lower_active_time_cols = [col for col in time_cols if df_lower_rows_filtered[col].max() >= N]

# 3. 최종 결과: 필터링된 행 + (기본정보 컬럼 + 데이터가 있는 시간대 컬럼)
df_lower_final = df_lower_rows_filtered[info_cols + df_lower_active_time_cols]

# 결과 확인 (퇴근 시간만 확인)
print(df_lower_final.iloc[:,[0,1,2,3,7,8]])

      호선         출발역 상하구분 요일구분     17     18
1    1호선         서울역   하선   평일   68.9  126.3
3    1호선          시청   하선   평일   66.0  111.9
129  3호선          지축   하선   평일   27.3   28.7
135  3호선          불광   하선   평일   21.2   20.3
137  3호선          녹번   하선   평일   21.6   21.6
139  3호선          홍제   하선   평일   22.9   22.3
141  3호선         무악재   하선   평일   23.1   22.7
143  3호선         독립문   하선   평일   23.9   23.9
145  3호선         경복궁   하선   평일   39.4   44.3
147  3호선          안국   하선   평일   55.1   65.8
211  4호선       미아사거리   하선   평일   24.8   27.3
213  4호선          길음   하선   평일   26.4   28.2
215  4호선      성신여대입구   하선   평일   32.1   33.2
217  4호선       한성대입구   하선   평일   35.4   36.2
219  4호선          혜화   하선   평일   45.3   50.2
221  4호선         동대문   하선   평일   51.7   51.5
247  4호선         남태령   하선   평일  100.9  104.3
269  5호선          목동   하선   평일   34.4   29.2
271  5호선         오목교   하선   평일   40.3   34.6
273  5호선          양평   하선   평일   43.1   38.2
275  5호선       영등포구청   하선   평일   31.0   27.1
277  5호선  

In [ ]:
[하선 혼잡도 100 별 조사]의 1호선(상선: 서울역 ~ 시청, 이 호선만 방향이 반대이다.)을 제외하고는 아래에서 위로 보는 것이 상선 방향이다. (하선은 위 -> 아래) 
- 상선 방향
ex) 3호선 (교대 ~ 지축), 4호선 (남태령 ~ 성신여대)
- 하선 방향
ex) 3호선 (지축 ~ 안국), 4호선 (미아사거리 ~ 남태령)

# 혼잡의 이유를 찾기 위해서는 퇴근 시간의 탑승객이 늘어나는 역을 주시해야 한다. (같은 맥락으로 하선에도 적용한다.)
- 주요 탑승객 상승 역(상승 폭 30)
4호선: 남태령(17, 18) - 환승
5호선: 길동(17, 18) - 제외(이유불명)
7호선: 고속터미널(17,18) - 환승 , 가산디지털단지(18) - 업무지구
8호선: 잠실(18) - 환승

In [ ]:
Insight
-  상/하선 혼잡도 120% 넘는 구간이 40곳이 존재함을 확인함으로써, 수도권 지하철의 혼잡도가 높다고 판단된다.
-  출/퇴근시간에 혼잡도가 높은 사실과 업무 지구 관련 지역(업무지구, 환승)에서 혼잡도가 높아지는 현상을 고려하였을때,
혼잡도가 높은 주요 이유로 과다하게 일자리가 집중되어 있음을 꼽을 수 있다. 다시 말해, 일자리 분산을 통해 혼잡도를 낮출 수 있다고 판단된다.

Limitation
아래 한계를 비롯하여 지하철과 함께 대중교통의 주축인 버스 혼잡도 등 추가사항을 조사하지 않는 이유는,
이미 참조(외부) 자료에서 수도권 대중교통 혼잡도 문제를 밝히고 있다고 판단되며, 검증의 과정으로서 짧은 조사로 마친다.
- 각 호선은 노선 별로 정리되어 있는데, 상선의 8호선의 복정이나 하선의 8호선 송파와 같이 각 호선 끝에 위치한 역의 경우,
특이 사항(혼잡률이 100보다 낮은 보이지 않는 노선에서 탑승객 상승 역)이 존재할 수 있다. 
- 2호선은 제외되었다. (칼럼"상하구분"이 내/외선 구분된다.)

In [ ]:
참조(외부) 자료

In [ ]:
대중교통 혼잡도 (정성적 자료)

1. 지하철
1.1
2024. "출근길 붐비는 서울지하철 4호선…한 칸은 의자 없이". 연합뉴스TV. https://www.youtube.com/watch?v=Bo9QPNEmZ6E.
1.2
권하영. 2024. "'출퇴근 시간 혼잡도 완화'...지하철 4·7호선 열차 2회씩 증회". 청년일보. https://www.youthdaily.co.kr/news/article.html?no=151373.
2. 버스
한우진. 2024. "계속 늘어나는 광역버스로 혼잡해지는 서울 도심, 해결 방안은?". 내손안에서울. https://mediahub.seoul.go.kr/archives/2011154.